In [1]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup as bs
import requests

In [2]:
densidad = pd.read_csv('../Data/clean_data/densidad_poblacion.csv')

In [3]:
densidad.head(1)

,prov_id,mun_id,total_2023,hombres_2023,mujeres_2023,ine_id,municipio,provincia,comunidad,km2,densidad
0,1,1001,2975.0,1539.0,1436.0,1001000000,Alegria-dulantzi,Provincia de Araba,Pais Vasco,1995.0,1.491228


In [42]:
contagio_mun = densidad.loc[:,['densidad', 'prov_id', 'mun_id']]

In [43]:
contagio_mun.head(2)

,densidad,prov_id,mun_id
0,1.491228,1,1001
1,1.070368,1,1002


In [44]:
#calculating zombie virus estimated basic reproduction number (transmision rate): infectiousnes * population density

c = 5 #constant representing infectiousness of the virus (For reference, flu is 1.3, measles is 12-18. We are picking 8 given that every human is susceptible of contracting the virus, there is no recovery and once bitten you are infected with a 100% probability.)

In [45]:
contagio_mun['R0'] = contagio_mun.densidad*c

In [46]:
contagio_mun['prob_inf'] = 1 - np.exp((-1*contagio_mun.R0*contagio_mun.densidad))

In [48]:
contagio_mun.head(3)

,densidad,prov_id,mun_id,R0,prob_inf
0,1.491228,1,1001,7.456140,0.999985
1,1.070368,1,1002,5.351842,0.996748
2,0.192302,1,1003,0.961512,0.168814


In [90]:
#contagio_mun.to_csv('contagio_municipios.csv', encoding='utf-8-sig', index = False)

In [54]:
contagio_prov = pd.DataFrame(densidad.prov_id)

In [55]:
contagio_prov.drop_duplicates(inplace = True)

In [81]:
contagio_prov['km2'] = contagio_prov.prov_id.apply(lambda x: sum(densidad[densidad.prov_id == x]['km2']))

In [86]:
contagio_prov['total_prov'] = contagio_prov.prov_id.apply(lambda x: sum(densidad[densidad.prov_id == x]['total_2023']))

In [105]:
contagio_prov['densidad'] = contagio_prov['total_prov'] / contagio_prov['km2']
contagio_prov.head(2)

,prov_id,km2,total_prov,densidad,R0,prob_inf
0,1,293470.0,336686.0,1.147259,1683430.0,1.0
51,2,1410921.0,387174.0,0.274412,967935.0,1.0


In [106]:
contagio_prov['R0'] = contagio_prov.densidad*c
contagio_prov['prob_inf'] = 1 - np.exp((-1*contagio_prov.R0*contagio_prov.densidad))

In [108]:
contagio_prov.head(3)

,prov_id,km2,total_prov,densidad,R0,prob_inf
0,1,293470.0,336686.0,1.147259,5.736293,0.998614
51,2,1410921.0,387174.0,0.274412,1.372061,0.313748
138,3,532101.0,1950357.0,3.665389,18.326944,1.000000


In [109]:
contagio_prov.to_csv('contagio_provincias.csv', encoding='utf-8-sig', index = False)